# **Model Evaluation**

Interactive notebook to evaluate and investigate performance of trained models

In [2]:
import os
import torch
from track_utils import *
from train_world import TrainWorld
from dataloader import TrackDataloader
from network import Net
from ppo import PPO
import matplotlib.pyplot as plt

%matplotlib inline

helper functions

In [3]:
from torch.distributions.categorical import Categorical

# inference function
def get_actions(observations, actor):
    """
        Obtains actions and logprobs from current observations
        Inputs:
            observations - (dict) maps track ids to (18x1) observation vectors
            actor - NN policy module
        Outputs:
            actions - (dict) maps track ids to discrete actions for observations
            logprobs -- (tesnor) log probabilities of each action
    """
    # handle initial frame where no observations are made
    if len(observations) == 0:
        return {}, []
    
    # apply policy on current observations 
    obs = torch.tensor(np.array(list(observations.values())).squeeze(), dtype=torch.float) # .to(self.device)
    logits = actor(obs)

    # get actions
    dist = Categorical(logits=logits)
    actions = dist.sample()

    # get logprob of each action
    logprobs = dist.log_prob(actions) # .detach()

    # map track IDs to actions
    try:
        actions = dict(zip(observations.keys(), actions.cpu().numpy()))
    except TypeError:
        # handle case for length 1 observation
        actions = dict(zip(observations.keys(), [actions.cpu().numpy().tolist()]))
        logprobs = logprobs.unsqueeze(0)
    
    return actions, logprobs

Get model

In [4]:
savepath = r"C:\Users\itber\Documents\learning\self_tutorials\CV_tracking\MARLMOT\trained_models"
policy_path = os.path.join(savepath, "actor_1161.pth")

In [5]:
actor = Net(input_dim=18, output_dim=5)
actor.load_state_dict(torch.load(policy_path))
actor.eval();

Get Data

In [6]:
# ground truth
TRAIN_FOLDERS = r"C:\Users\itber\Documents\datasets\MOT15\train"
dataloader = TrackDataloader(TRAIN_FOLDERS)

select video

In [15]:
ground_truth, detections, frame_size = dataloader.__getitem__(0)

tracker = HungarianTracker(iou_threshold=0.3, min_age=1)
world = TrainWorld(tracker=tracker, 
                    ground_truth=ground_truth, 
                    detections=detections,
                    frame_size=frame_size)

iterate through video

In [16]:
observation_list = []
action_list = []
rewards_list = []
logprob_list = []

batch_length = []

observations, rewards, done = world.step({})

while True:
    
    # append observations
    observation_list.append(observations)
    
    # get actions and step
    actions, logprobs = get_actions(observations, actor)
    observations, rewards, done = world.step(actions)

    rewards_list.append(rewards)
    action_list.append(actions)
    logprob_list.append(logprobs)

    batch_length += list(actions.values())

    if done:
        break

Inspect rewards

In [8]:
total_rewards = 0
mean_rewards = []
for rew in rewards_list:
    total_rewards += sum(rew)
    mean_rewards.append(np.mean(rew))

total_rewards

c:\Users\itber\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\itber\anaconda3\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


-3942.468823671341

Investigate actions

In [27]:
action_list

[{},
 {},
 {0: 2, 1: 2, 2: 2, 3: 2, 4: 2},
 {0: 2, 1: 2, 2: 2, 3: 2, 4: 2},
 {0: 2, 1: 2, 3: 2, 6: 2, 7: 2},
 {0: 2, 1: 2, 6: 2, 8: 2},
 {1: 2, 6: 2, 8: 2},
 {1: 2, 6: 2, 8: 2, 9: 2},
 {1: 2, 6: 2, 8: 2},
 {1: 2, 6: 2, 8: 2, 15: 2},
 {1: 2, 8: 2, 15: 2, 16: 2, 17: 2, 18: 2},
 {1: 2, 15: 2, 16: 2, 18: 2},
 {1: 2, 16: 2, 19: 2},
 {1: 2, 16: 2, 22: 2},
 {1: 2, 16: 2, 22: 2, 25: 2},
 {1: 2, 16: 2, 22: 2, 25: 2, 27: 2},
 {1: 2, 16: 2, 22: 2, 27: 2, 28: 2},
 {1: 2, 16: 2, 22: 2, 27: 2},
 {1: 2, 16: 2, 22: 2, 27: 2},
 {1: 2, 16: 2, 22: 2, 27: 2, 30: 2, 31: 2},
 {1: 2, 16: 2, 22: 2, 27: 2, 30: 2, 31: 2},
 {1: 2, 16: 2, 22: 2, 27: 2, 31: 2},
 {1: 2, 16: 2, 22: 2, 27: 2, 31: 3},
 {1: 2, 16: 2, 22: 2, 27: 2, 31: 2, 33: 2},
 {1: 2, 16: 2, 22: 2, 27: 2, 31: 2, 33: 2},
 {1: 2, 16: 2, 22: 2, 27: 2, 31: 2, 33: 2},
 {1: 2, 16: 2, 22: 2, 27: 2, 31: 2, 35: 2, 36: 2},
 {1: 2, 16: 2, 22: 2, 31: 2, 36: 2},
 {1: 2, 22: 2, 31: 4, 36: 2},
 {1: 2, 22: 2, 31: 2, 42: 2},
 {1: 2, 22: 2, 31: 2, 42: 2, 44: 2},
 {1: 

In [10]:
actions = []
for a in action_list:
    actions += list(a.values())

print(len(action_list))
np.unique(actions, return_counts=True)

525


(array([1, 2, 3, 4], dtype=int64),
 array([   1, 3809,   38,   17], dtype=int64))

In [11]:
world.truth_tracks

,frame,id,bb_left,bb_top,bb_width,bb_height
4999,525,8,1673,443,61,193
5000,525,9,767,333,137,465
5001,525,10,713,299,253,577
5002,525,11,1601,401,89,275
5003,525,14,1429,459,95,199
5004,525,17,1847,433,71,185
5005,525,18,1805,423,113,237
5006,525,20,1271,447,87,211
5007,525,21,1339,439,83,225
5008,525,23,1627,453,77,193


In [12]:
world.id_map

{977: 8, 987: 10, 1024: 11, 1026: 14, 1027: 18, 1016: 20}

In [13]:
for track in world.tracker.tracks:
    print(track.id, track.age, track.track_mode, track.get_state())
    print(track.time_since_update, track.hit_streak)

977 24 1 [[1661.85814093  435.44212473 1751.45256135  638.56358612]]
1 23
987 18 1 [[705.82531789 269.461754   948.60439283 820.20891539]]
1 17
1016 4 1 [[1274.57629316  434.30427413 1371.04222986  652.89292923]]
1 3
1024 2 1 [[1602.  403. 1715.  660.]]
1 1
1026 1 1 [[1430.  465. 1482.  583.]]
1 0
1027 1 1 [[1820.  381. 1943.  661.]]
1 0
1029 0 1 [[1806.  521. 1843.  604.]]
0 0
1030 0 1 [[1479.  580. 1507.  644.]]
0 0


In [14]:
world.tracker.tracks[0].id

977

In [15]:
world.tracker.tracks[0].history

[array([[1661.85814093,  435.44212473, 1751.45256135,  638.56358612]])]